# 1DBLM Fortran Demo

This notebook compiles the Fortran module and driver, runs a synthetic case, and plots KM/KH after applying the correction.

In [ ]:
# Compile Fortran sources (requires gfortran on PATH)
import subprocess, sys
src_dir = '../implementations'
cmd = ['gfortran', '-O2', '-std=f2008', '-o', 'mcnb_driver',
       f'{src_dir}/McNider_1DBLM_fc_module.f90', f'{src_dir}/mcnb_fc_driver.f90']
print('Compiling...')
proc = subprocess.run(' '.join(cmd), shell=True, stdout=subprocess.PIPE, stderr=subprocess.PIPE, text=True)
if proc.returncode != 0:
    print('Compile failed:\n', proc.stderr)
else:
    print('Compiled executable: mcnb_driver')

In [ ]:
# Run the driver and capture stdout
import subprocess, io
proc = subprocess.run(['./mcnb_driver'], stdout=subprocess.PIPE, stderr=subprocess.PIPE, text=True)
if proc.returncode != 0:
    print('Error running driver:\n', proc.stderr)
else:
    out = proc.stdout.strip().splitlines()
    print('\n'.join(out[:10]))  # show first lines

In [ ]:
# Parse the printed CSV rows and plot KM/KH
import matplotlib.pyplot as plt
rows = [r.strip() for r in out if r.strip() and (r.strip()[0].isdigit())]
z = []; z_g = []; km = []; kh = []
for r in rows:
    parts = r.split()
    # driver prints: level z z_g km kh
    z.append(float(parts[1])); z_g.append(float(parts[2])); km.append(float(parts[3])); kh.append(float(parts[4]))

plt.figure(figsize=(6,4))
plt.plot(km, z, 'o-', label='KM (corrected)')
plt.plot(kh, z, 's--', label='KH (corrected)')
plt.gca().invert_yaxis()
plt.xlabel('K (m2/s)'); plt.ylabel('z (m)')
plt.legend(); plt.grid(True); plt.title('Fortran driver: corrected diffusivities')
plt.show()